In [1]:
import numpy as np
import nest 

/home/winkrobed/miniconda3/envs/nest_33_ehp/lib/python3.10/site-packages/nest/ll_api.py:42: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  import scipy



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.3
 Built: Sep 12 2022 04:40:40

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
module_name = "edlif_psc_alpha_0_module"
nest.Install(module_name)

neuron_model_name = "edlif_psc_alpha_percent0_nestml__with_ed_stdp0_nestml"
synapse_model_name = "ed_stdp0_nestml__with_edlif_psc_alpha_percent0_nestml"


May 24 13:47:13 Install [Info]: 
    loaded module edlif_psc_alpha_0_module


In [3]:
verbose = True

In [4]:
init_w = 1
resolution = 1
delay = 1.

fix_atp = 0  # 1 or 0
atp = 100

neuron_parameters = {
    "ATP": atp if fix_atp == 1 else 100, 
    "fix_atp": fix_atp
}

lmbda = 1E-6
eta = 5

synapse_parameters={
    "alpha": 0.5,
    #"lambda": 0.005,
    "mu_plus": 0,
    "mu_minus": 0,
    "eta": eta
}

sim_time = 5.  # [ms]
total_time = 1000
epochs = total_time//sim_time
pre_spike_time = 100. #sim_time / 2  # [ms]
post_spike_time = 200. #sim_time / 2  # [ms]

In [5]:
nest.set_verbosity("M_WARNING")
#nest.set_verbosity("M_ALL")

if verbose:
    print(f'initial weight: {init_w}')

nest.ResetKernel()
nest.SetKernelStatus({'resolution': resolution})

wr = nest.Create('weight_recorder')
nest.CopyModel(
    synapse_model_name, 
    "stdp_nestml_rec",
    {
        "weight_recorder": wr[0],
        "w": init_w,
        "delay": delay,
        "d": delay,
        "receptor_type": 0
    })

# create spike_generators with these times
pre_sg = nest.Create(
    "spike_generator",
    params={
        "spike_times": [pre_spike_time, total_time - 10.]})
post_sg = nest.Create(
    "spike_generator",
    params={
        "spike_times": [10, 50, 100, 150, 200],
        'allow_offgrid_times': True})

# create parrot neurons and connect spike_generators
pre_neuron = nest.Create("parrot_neuron")
post_neuron = nest.Create(neuron_model_name)
if neuron_parameters is not None:
    for param, param_value in neuron_parameters.items():
        if verbose:
            print(f'setting param: {param} -> {param_value}')
        post_neuron.set({param: param_value})

spikedet_pre = nest.Create("spike_recorder")
spikedet_post = nest.Create("spike_recorder")
##mm = nest.Create("multimeter", params={"record_from" : ["V_m"]})

nest.Connect(
    pre_sg, pre_neuron, 
    "one_to_one", 
    )
nest.Connect(
    post_sg, post_neuron, 
    "one_to_one", 
    syn_spec={"weight": 1209.}) #1599.
nest.Connect(
    pre_neuron, post_neuron, 
    "all_to_all", 
    syn_spec={'synapse_model': 'stdp_nestml_rec'})
##nest.Connect(mm, post_neuron)

nest.Connect(pre_neuron, spikedet_pre)
nest.Connect(post_neuron, spikedet_post)

# get STDP synapse and weight before protocol
syn = nest.GetConnections(
    source=pre_neuron, 
    synapse_model="stdp_nestml_rec")
if synapse_parameters is None:
    synapse_parameters = {}
synapse_parameters.update({"lambda": lmbda})
synapse_parameters.update({"eta": eta})
nest.SetStatus(syn, synapse_parameters)

# Initial weight
initial_weight = nest.GetStatus(syn)[0]["w"]
np.testing.assert_allclose(initial_weight, init_w)

# Simulate
for i in range(int(epochs)):
    nest.Simulate(sim_time)

# Updated weight
updated_weight = nest.GetStatus(syn)[0]["w"]

# Get spike times
actual_t_pre_sp = nest.GetStatus(spikedet_pre)[0]["events"]["times"][0]
actual_t_post_sp = nest.GetStatus(spikedet_post)[0]["events"]["times"][0]

# Calculate dt and dw
dt = actual_t_post_sp - actual_t_pre_sp
dw = (updated_weight - initial_weight) / lmbda

if verbose:
    print("prespike time")
    print(nest.GetStatus(spikedet_pre)[0]["events"]["times"])
    print("prespike senders")
    print(nest.GetStatus(spikedet_pre)[0]["events"]["senders"])
    print("postspike time")
    print(nest.GetStatus(spikedet_post)[0]["events"]["times"])
    print("postspike senders")
    print(nest.GetStatus(spikedet_post)[0]["events"]["senders"])
    print("initial weight")
    print(initial_weight)
    print("updated weight")
    print(updated_weight)

initial weight: 1
setting param: ATP -> 100
setting param: fix_atp -> 0
prespike time
[101. 991.]
prespike senders
[4 4]
postspike time
[ 17.  57. 107. 157. 207.]
postspike senders
[5 5 5 5 5]
initial weight
1.0
updated weight
1.000070115627295
